In [1]:
import ollama 

In [2]:
import json
from datetime import datetime

In [9]:
# Expanded Mock DB
# --- FRENCH INVENTORY ---
inventory_db = {
    "ordinateur": {"stock": 5, "base_price": 1200, "weight_kg": 2.5},
    "ecran": {"stock": 12, "base_price": 300, "weight_kg": 5.0},
    "clavier": {"stock": 25, "base_price": 80, "weight_kg": 0.5},
    "iphone": {"stock": 8, "base_price": 999, "weight_kg": 0.2}
}

# Create Tools function

In [10]:
# --- TOOL 1: Inventory ---
def check_inventory(product_name):
    return inventory_db.get(product_name.lower(), {"error": "Product not found"})

# --- TOOL: Get TVA (VAT) ---
def get_tva_rate(region_name):
    """Returns the TVA rate for French regions. Standard is 20%."""
    # Simplified logic: Most of mainland France is 20%
    # Corsica (Corse) or Overseas might have different rates
    regional_rates = {
        "corse": 0.10, 
        "guadeloupe": 0.085,
        "martinique": 0.085
    }
    rate = regional_rates.get(region_name.lower(), 0.20)
    return {"region": region_name, "tva_rate": rate}

# --- TOOL: Shipping (Metric System) ---
def calculate_shipping_fr(weight_kg, distance_km):
    """Standard EU shipping: €10 base + €1.50 per kg."""
    cost = 10 + (weight_kg * 1.5) + (distance_km * 0.05)
    return {"shipping_cost_eur": round(cost, 2)}

available_functions = {
    'check_inventory': check_inventory, 
    'calculate_shipping': calculate_shipping_fr,
    'get_tva_rate': get_tva_rate
}

# Create Advanced Tools Schema 
Define the schemas so the LLM knows it needs the from the inventory tool to use the shipping tool.

In [11]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "check_inventory",
            "description": "Retrieve price, stock, and weight in kg for a product",
            "parameters": {
                "type": "object",
                "properties": {"product_name": {"type": "string"}},
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate_shipping",
            "description": "Calculate shipping costs in Euros for France based on kg and km",
            "parameters": {
                "type": "object",
                "properties": {
                    "weight_kg": {"type": "number"},
                    "distance_km": {"type": "number"}
                },
                "required": ["weight_kg", "distance_km"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_tva_rate",
            "description": "Get the French TVA (VAT) rate for a specific region (e.g., Bretagne, Corse, PACA)",
            "parameters": {
                "type": "object",
                "properties": {"region_name": {"type": "string"}},
                "required": ["region_name"]
            }
        }
    }
]

In [13]:
def run_query(user_prompt):
    messages = [{"role": "user", "content": user_prompt}]
    
    # Primary Call
    response = ollama.chat(model="qwen3:8b", messages=messages, tools=tools)
    
    # Recursive Tool Handling
    while response['message'].get('tool_calls'):
        messages.append(response['message'])
        
        for tool in response['message']['tool_calls']:
            name = tool['function']['name']
            args = tool['function']['arguments']
            
            # Map correctly to our French functions
            result = available_functions[name](**args)
            messages.append({'role': 'tool', 'content': json.dumps(result)})
            
        # Check if more tools are needed after receiving results
        response = ollama.chat(model="qwen3:8b", messages=messages, tools=tools)
    
    return response['message']['content']



In [14]:
# Example Complex Query
query = "Je suis à Marseille (800km). Quel est le prix total pour un 'ordinateur' avec la TVA et les frais de port ?"
print(run_query(query))



Le prix total pour un ordinateur à Marseille, incluant la TVA et les frais de port, est de **1 504,50 €**.

**Détails :**
- **Prix du produit** : 1 200 €
- **Frais de port** : 53,75 € (calculés sur 800 km et 2,5 kg)
- **TVA (PACA)** : 20 % sur le montant total (1 200 € + 53,75 € = 1 253,75 €)
- **TVA calculée** : 250,75 €
- **Total** : 1 253,75 € + 250,75 € = **1 504,50 €**
